 * config.json  with 
```text
{
  "GEMINI_API_KEY": "Your Key",
  "KIMI_API_KEY":  "Your Key",
  "HTTP_PROXY_URL": "http://localhost:port"
} 
```

In [ ]:
# funct
import os
import gradio as gr
import traceback
import json
from openai import OpenAI
import google.generativeai as genai

# Function to check if a proxy is available
import socket

# Define the is_proxy_available function at the top of your script
def is_proxy_available(url):
    """Checks if a proxy is available by trying to connect to its host and port."""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)  # Timeout after 1 second

        # --- FIX IS HERE ---
        # The host and port must be passed as a single tuple: (host, port)
        address = (url["host"], url["port"])
        result = sock.connect_ex(address)
        # --- END OF FIX ---

        if result == 0:
            # A result of 0 means the connection was successful
            return True
    except socket.error as err:
        # This can happen for various reasons (e.g., DNS resolution failure)
        print(f"Socket error while checking proxy: {err}")
    except KeyError:
        # Handle cases where the url dict might be missing 'host' or 'port'
        print(f"Invalid URL dictionary passed to is_proxy_available: {url}")
    finally:
        # Always close the socket to free up resources
        sock.close()
        
    return False
# --- Global Configuration ---
CONFIG = {}

# --- Global Variables ---
kimi_client = None
gemini_model = None
current_model_technical_name = None
gemini_configured = False
kimi_configured = False

AVAILABLE_MODELS = {
    # --- Kimi (Moonshot AI) Models ---
    # Using the stable, documented v1 models.
    "Kimi (32k context)": "moonshot-v1-32k",
    "Kimi (128k context)": "moonshot-v1-128k",

    # --- Google Gemini Models ---
    # The 'latest' tag ensures you get the most recent version automatically.
    "Gemini 1.5 Flash": "gemini-1.5-flash-latest",
    "Gemini 2.5 Flash": "gemini-2.5-flash-latest",
    "Gemini 1.5 Pro": "gemini-1.5-pro-latest",
    "Gemini 2.5 Pro": "gemini-2.5-pro-latest",
    
}

DEFAULT_MODEL_DISPLAY_NAME = "Kimi (32k context)"

def load_and_configure():
    """
    Loads configuration from config.json and sets up the proxy globally.
    This is the one-stop function for all initial setup.
    """
    global CONFIG
    try:
        # Load from config.json in the current directory (for Jupyter)
        config_path = os.path.join(os.getcwd(), 'config.json')
        print(f"Attempting to load configuration from: {config_path}")
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                CONFIG = json.load(f)
            print("Successfully loaded settings from config.json")
        else:
            print("WARNING: config.json not found.")
            CONFIG = {}
    except Exception as e:
        print(f"ERROR: Could not read or parse config.json: {e}")
        CONFIG = {}

    # Allow environment variables to override the config file
    CONFIG['GEMINI_API_KEY'] = os.environ.get("GEMINI_API_KEY", CONFIG.get('GEMINI_API_KEY'))
    CONFIG['KIMI_API_KEY'] = os.environ.get("KIMI_API_KEY", CONFIG.get('KIMI_API_KEY'))
    CONFIG['HTTP_PROXY_URL'] = os.environ.get("HTTP_PROXY_URL", CONFIG.get('HTTP_PROXY_URL'))

    # --- SET PROXY GLOBALLY FOR ALL LIBRARIES ---
    proxy_url = CONFIG.get("HTTP_PROXY_URL")
    if proxy_url:
        # Parse the proxy URL to get the host and port
        proxy_parts = proxy_url.split('://')[-1].split(':')
        host = proxy_parts[0]
        port = int(proxy_parts[-1])
        if is_proxy_available({"host": host, "port": port}):
            os.environ['http_proxy'] = proxy_url
            os.environ['https_proxy'] = proxy_url
            print(f"Proxy set globally for this session: {proxy_url}")
        else:
            print("Proxy is not available or not configured. Using direct connection.")
            # Remove proxy environment variables if they exist
            os.environ.pop('http_proxy', None)
            os.environ.pop('https_proxy', None)
    else:
        print("No proxy configured. Using direct connection.")
        # Remove proxy environment variables if they exist
        os.environ.pop('http_proxy', None)
        os.environ.pop('https_proxy', None)

    # --- CRITICAL: Set NO_PROXY for Gradio's internal communication ---
    current_no_proxy = os.environ.get('NO_PROXY', '')
    additional_no_proxy_hosts = ['localhost', '127.0.0.1', '0.0.0.0']
    new_no_proxy_parts = [host for host in current_no_proxy.split(',') if host.strip()]
    for host in additional_no_proxy_hosts:
        if host not in new_no_proxy_parts:
            new_no_proxy_parts.append(host)
    os.environ['NO_PROXY'] = ','.join(new_no_proxy_parts)
    print(f"NO_PROXY set for Gradio: {os.environ.get('NO_PROXY')}")


def initialize_api_clients():
    """Initializes the API clients using the globally set proxy."""
    global kimi_client, gemini_configured, kimi_configured
    
    # Initialize Gemini
    gemini_api_key = CONFIG.get('GEMINI_API_KEY')
    if gemini_api_key:
        try:
            # Let Gemini auto-detect the proxy from the environment
            genai.configure(api_key=gemini_api_key)
            gemini_configured = True
            print("Google Gemini API configured successfully.")
        except Exception as e:
            print(f"ERROR: Failed to configure Gemini API: {e}")
    else:
        print("INFO: GEMINI_API_KEY not found.")

    # Initialize Kimi
    kimi_api_key = CONFIG.get('KIMI_API_KEY')
    if kimi_api_key:
        try:
            # Let Kimi auto-detect the proxy from the environment
            kimi_client = OpenAI(
                api_key=kimi_api_key,
                base_url="https://api.moonshot.cn/v1",
            )
            kimi_configured = True
            print("Kimi (Moonshot) client initialized successfully.")
        except Exception as e:
            print(f"ERROR: Failed to initialize Kimi client: {e}")
    else:
        print("INFO: KIMI_API_KEY not found.")


# --- Chat Responder Functions (Unchanged) ---
def gemini_chat_responder(user_message, chat_history):
    global gemini_model, current_model_technical_name
    if gemini_model is None or gemini_model.model_name != f"models/{current_model_technical_name}":
        gemini_model = genai.GenerativeModel(current_model_technical_name)
    gemini_history = [{'role': 'user' if item[0] else 'model', 'parts': [item[0] if item[0] else item[1]]} for item in chat_history]
    try:
        chat_session = gemini_model.start_chat(history=gemini_history)
        response = chat_session.send_message(user_message)
        return response.text
    except Exception as e:
        traceback.print_exc(); return f"An error occurred with Google Gemini: {e}"

def kimi_chat_responder(user_message, chat_history):
    messages = [{"role": "system", "content": "你是 Kimi，由 Moonshot AI 出品的超长上下文人工智能助手。"}]
    for user_msg, ai_msg in chat_history:
        messages.extend([{'role': 'user', 'content': user_msg}, {'role': 'assistant', 'content': ai_msg}])
    messages.append({'role': 'user', 'content': user_message})
    try:
        response = kimi_client.chat.completions.create(model=current_model_technical_name, messages=messages, temperature=0.7)
        return response.choices[0].message.content
    except Exception as e:
        traceback.print_exc(); return f"An error occurred with the Kimi API: {e}"

def master_chat_responder(user_message, chat_history):
    global current_model_technical_name
    if not user_message.strip(): return "Please type a message."
    if 'moonshot' in current_model_technical_name:
        if not kimi_configured: return "Error: Kimi API key not configured."
        return kimi_chat_responder(user_message, chat_history)
    elif 'gemini' in current_model_technical_name:
        if not gemini_configured: return "Error: Gemini API key not configured."
        return gemini_chat_responder(user_message, chat_history)
    else:
        return "Error: Unknown model type."

# --- Gradio UI (Unchanged) ---
with gr.Blocks(theme="soft", title="Multi-LLM Chat") as demo:
    gr.Markdown("# Multi-LLM Chat Dashboard (Kimi & Gemini)")
    
    # --- Run ALL setup first ---
    load_and_configure()
    initialize_api_clients()
    
    current_model_technical_name = AVAILABLE_MODELS.get(DEFAULT_MODEL_DISPLAY_NAME)
    with gr.Row():
        model_dropdown = gr.Dropdown(choices=list(AVAILABLE_MODELS.keys()), value=DEFAULT_MODEL_DISPLAY_NAME, label="Select LLM Model")
        model_status_text = gr.Textbox(label="Model Status", value=f"Current: {current_model_technical_name}", interactive=False)
    chat_interface = gr.ChatInterface(fn=master_chat_responder, chatbot=gr.Chatbot(height=500), retry_btn="Retry", undo_btn="Undo", clear_btn="Clear")
    def handle_model_change(selected_display_name):
        global current_model_technical_name, gemini_model
        technical_name = AVAILABLE_MODELS.get(selected_display_name)
        if 'moonshot' in technical_name and not kimi_configured: return "Cannot switch: Kimi API not configured."
        if 'gemini' in technical_name and not gemini_configured: return "Cannot switch: Gemini API not configured."
        if current_model_technical_name and 'gemini' in current_model_technical_name: gemini_model = None
        current_model_technical_name = technical_name
        return f"Switched to model: {current_model_technical_name}"
    model_dropdown.change(fn=handle_model_change, inputs=[model_dropdown], outputs=[model_status_text])
    if not gemini_configured: gr.Markdown("<p style='color:orange;'>Warning: Gemini API not configured.</p>")
    if not kimi_configured: gr.Markdown("<p style='color:orange;'>Warning: Kimi API not configured.</p>")

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=False)